# ETL Pipeline

This notebook will use disaster response message data from [Figure 8](https://www.figure-eight.com/) to create an ETL pipeline.

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## Assess

### Messages

In [3]:
messages = pd.read_csv('data/disaster_messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
messages.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

**Oberservations:** 
- `original` column are in French so we need to drop it because we anlayze English message.
- `genre` column only got 3 types which is direct, social and news

### Categories

In [5]:
categories = pd.read_csv('data/disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [14]:
#check the categories in the first row
categories.categories.iloc[0]

'related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'

In [18]:
#check got how many items in categories
len(categories.categories.iloc[0].split(';'))

36

**Observations:**
- categories got 36 items which mean possible got 36 classes to be predicted with machine learning model.

## Clean

In [8]:
messages_clean = messages.copy()
categories_clean = categories.copy()

### Categories

**1. Create new df of categories with category name for heading and 1s and 0s for values**

- Use `.str.split(';', expand=True)` to split data
- Use `.str[:-2]` to get headings
- Use `.str[-1]` and `.astype(int)` to convert data to 1s and 0s

In [19]:
# Create df
df = categories.categories.str.split(';', expand=True)

In [21]:
# extract a list of new column names for categories by select every char except the last 2
col_names = df.loc[0].str[:-2]
df.columns = col_names
df.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object', name=0)

In [22]:
# set each value to be the last character of the string and convert into integer
for col in col_names:
    df[col] = df[col].str[-1].astype(int)

In [23]:
df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**2. Replace `categories` column with df**
- Use `pd.concat([categories_clean, df], axis=1)` & `.drop('categories', axis=1)`

In [24]:
#concatenate the original dataframe(categories) with the new dataframe (df)
#drop duplicates
categories_clean = pd.concat([categories.drop('categories', axis=1), df], axis=1)
categories_clean.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**3. Create merged dataset**
- Use `.merge(on='id')`

The `original` column and `id` column were retained for storage in the database even if these will not be used in the machine learning process.

In [25]:
df = messages.merge(categories_clean, on='id')

In [27]:
#check duplicates items
df[df.duplicated()]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
164,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
165,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
658,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
659,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
660,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
716,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1415,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1416,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2139,2446,How much money did TV Latino American collect ...,Konbyen kob tv latino america ranmase miami,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**4. Drop duplicated rows.**
- Use `.drop_duplicates(inplace=True)`

In [28]:
#drop duplicates items
df.drop_duplicates(inplace=True)

**5. Explore values for categories**

In [30]:
#check the unique items for column start from 4th column which is related column
for col in df.columns[4:]:
    if (df[col] > 1).sum() > 0:
        print(col)
        print(df[col].unique())
        print('\n')

related
[1 0 2]




`related` column got 3 different values. It supposed be only 0 and 1. So 2 either remove or convert to 1.
the total number of related = 2 total got 188 so I decided to remove these rows.

In [31]:
(df.related == 2).sum()

188

In [33]:
# remove 'related = 2'
df = df[(df['related'] == 1) | (df['related'] == 0)]

In [31]:
#convert 'related = 2' to 'related = 1'
#df.loc[(df.related == 2), 'related'] = 1

## Load

In [34]:
engine = create_engine('sqlite:///disaster_messages.db')
df.to_sql('messages', engine, index=False, chunksize=1000, if_exists='replace') 